In [1]:
import re
import string
import numpy as np
import math
import nltk
import io
import csv
import pandas as pd
import itertools
import nltk
import wikiwords
import argparse
import re
import requests
import pattern
import syllables
import json
import enchant
import gensim 
import random
import gensim.downloader as api


from collections import defaultdict
from nltk.corpus import brown
from difflib import SequenceMatcher
from textblob import TextBlob
from autocorrect import Speller
from enchant.checker import SpellChecker
from array import array
from sklearn.metrics.pairwise import cosine_similarity
from wordfreq import zipf_frequency
from wikiwords import occdb
from nltk.corpus import wordnet
from py_thesaurus import Thesaurus
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import islice
from pattern.text.en import pluralize, singularize, comparative, superlative, conjugate
from pattern.text.en import tenses, INFINITIVE, PRESENT, PAST, FUTURE

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')
nltk.download('brown')

[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/apple/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/apple/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /Users/apple/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package brown to /Users/apple/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [3]:
wiki_freq_dict = {}
lexicon_dict = {}
complex_words = []
BIGHUGE_KEY = "105a58f9d880af14af1ca1abf6b1f996"
word_vectors = api.load("glove-wiki-gigaword-300")
ppdb_filepath = "ppdb-2.0-m-lexical"

In [4]:
lemmatizer = WordNetLemmatizer()

In [5]:
def is_similar(word1, word2):
    similarity_ratio = SequenceMatcher(None, word1, word2).ratio()
    return similarity_ratio >= 0.7

In [6]:
# Function used for flattening list of lists to a single list
def flatten(l):
    return [item for sublist in l for item in sublist]

In [7]:
def generateWikiFreqDict(text_file):
    wiki_freq_dict = {}
    with open(text_file) as f:
        for line in f.readlines():
            (word, freq) = line.split()
            wiki_freq_dict[word.lower()] = freq
    return wiki_freq_dict

In [8]:
wiki_freq_dict = generateWikiFreqDict("wiki_frequencies.txt")
N = 30
out = dict(itertools.islice(wiki_freq_dict.items(), N))
print(out)

{'the': '183212978', 'of': '86859699', 'in': '75290639', 'and': '74708386', 'a': '53698262', 'to': '52250362', 'was': '32540285', 'is': '23812199', 'on': '21691194', 'for': '21634075', 'as': '21126503', 'with': '18605836', 'by': '18143981', 'he': '16395831', 'that': '14608114', 'at': '14470966', 'from': '13716922', 'his': '12708933', 'it': '11862812', 'an': '9988978', 'were': '8516652', 'which': '7655695', 'are': '7321844', 'also': '6466616', 'this': '6427119', 'had': '6236977', 'be': '6147211', 'first': '5897144', 'or': '5763356', 'has': '5757604'}


In [9]:
def generateLexiconDict(tsv_file):
    lexicon_dict = {}
    with open(tsv_file) as f:
        file = csv.reader(f, delimiter="\t")
        for line in file:
            word = line[0]
            score = line[1]
            lexicon_dict[word.lower()] = score
    return lexicon_dict

In [10]:
lexicon_dict = generateLexiconDict('lexicon.tsv')

In [11]:
# LS step1: Complex Word Identification

def word_preceding(text, word):
    words = text.split()
    index = words.index(word)
    return words[index-1] if index > 0 else None

def word_following(text, word):
    words = text.split()
    index = words.index(word)
    return words[index+1] if index+1 < len(words) else None

def complexWordIdentif(article):
    threshold_scores_dict = {}
    word_sentence_dict = {}
    for sentence in sent_tokenize(article):
        for word_ in word_tokenize(sentence):
            word = word_.lower()
            word_split_hyphen = word_.split("-")
            word_split_underscore = word_.split("_")
            if(len(word_split_hyphen)>1):
                total_freq = 0
                total_lexicon = 0
                for word_hyph in word_split_hyphen:
                    total_freq+=int(wiki_freq_dict.get(word_hyph,0))
                    total_lexicon+=float(lexicon_dict.get(word_hyph,0))
                wiki_freq = int(total_freq/len(word_split_hyphen))
                lexicon_score = float(total_lexicon/len(word_split_hyphen))
                if(wiki_freq<12000 or lexicon_score>3.0):
                    threshold_scores_dict[word]=1
                    word_sentence_dict[word] = sentence
                    
            elif(len(word_split_underscore)>1):
                total_freq = 0
                total_lexicon = 0
                for word_un in word_split_underscore:
                    total_freq+=int(wiki_freq_dict.get(word_un,0))
                    total_lexicon+=float(lexicon_dict.get(word_un,0))
                wiki_freq = int(total_freq/len(word_split_underscore))
                lexicon_score = float(total_lexicon/len(word_split_underscore))
                if(wiki_freq<12000 or lexicon_score>3.0):
                    threshold_scores_dict[word]=1
                    word_sentence_dict[word] = sentence

            else:
                threshold_scores_dict[word]=0
                if(word in wiki_freq_dict and word in lexicon_dict):
                    wiki_freq = int(wiki_freq_dict[word])
                    lexicon_score = float(lexicon_dict[word])
    #                 print(word, wiki_freq, lexicon_score)
                    if(wiki_freq<12000 or lexicon_score>3.0):
                        threshold_scores_dict[word]=1
                        word_sentence_dict[word] = sentence

                elif(word in wiki_freq_dict):
                    wiki_freq = int(wiki_freq_dict[word])
    #                 print(word, wiki_freq)
                    if(wiki_freq<12000):
                        threshold_scores_dict[word]=1
                        word_sentence_dict[word] = sentence
                        prev_word = word_preceding(sentence, word)
                        next_word = word_following(sentence, word)
                        if(prev_word and next_word):
                            three_gram_dict[word] = [prev_word, word, next_word]
                elif(word in lexicon_dict):
                    lexicon_score = float(lexicon_dict[word])
    #                 print(word, lexicon_score)
                    if(lexicon_score>3.0):
                        threshold_scores_dict[word]=1
                        word_sentence_dict[word] = sentence
                        prev_word = word_preceding(sentence, word)
                        next_word = word_following(sentence, word)
                        if(prev_word and next_word):
                            three_gram_dict[word] = [prev_word, word, next_word]
    return threshold_scores_dict, word_sentence_dict

In [12]:
# Get the pos tag of a certain word 

def getPosTag(word):
    tag = nltk.pos_tag([word])
    return tag[0][1]

def getPosTagFromSentence(string, target_word):
    pos_tag=""
    tokens = nltk.word_tokenize(string)
    tag = nltk.pos_tag(tokens)
    for pair in tag:
        if(pair[0]==target_word):
            pos_tag = pair[1]
    return pos_tag

In [13]:
# Used with word net dictionary

def getTypeFromTag(tag):
    # Convert all forms of noun tags to "n" noun type
    if(tag=="NN" or tag=="NNS" or tag=="NNP" or tag=="NNPS"):
        return 'n'
    # Convert all forms of adjective tags to "a" adjective type
    elif(tag=="JJ" or tag=="JJR" or tag=="JJS"):
        return 'a'
    # Convert all forms of verb tags to "v" verb type
    elif(tag=="VBZ" or tag=="VB" or tag=="VBP" or tag=="VBN" or tag=="VBG" or tag=="VBD"):
        return 'v'
    # Convert all forms of adverb tags to "r" adverb type
    elif(tag=="RBS" or tag=="RB" or tag=="RBR"):
        return 'r'
    else:
        return tag

In [14]:
# Word Net Synonyms

def getSynWordNet(complex_word):
    synonyms = []
    for synset in wordnet.synsets(complex_word):
            for l in synset.lemmas():
                synonyms.append(l.name())
    return list(set(synonyms))

In [15]:
# Word Net Synonyms with specific pos tag

def getSynWordNetSpec(complex_word, pos_tag):
    synonyms = []
    for synset in wordnet.synsets(complex_word):
        if(synset.pos()==pos_tag):
            for l in synset.lemmas():
                synonyms.append(l.name())
    return list(set(synonyms))

In [16]:
# BigHuge thesaurus

def getSynBigHuge(complex_word):
    bighuge_synonyms = []
    r = requests.get(url='http://words.bighugelabs.com/api/2/'+BIGHUGE_KEY+'/'+complex_word+'/json')  
    if(r.status_code!=404 and r.status_code!=500):
#         print(type(r.json()),"\n",r.json())
        if(type(r.json()) is dict):
            synonym_dict = r.json()
            for key in synonym_dict: # key may be: noun/verb/adjective/adverb
                synonym_list = synonym_dict[key].get("syn")
                if(synonym_list):
                    bighuge_synonyms.append(synonym_list)
            flatList = [element for innerList in bighuge_synonyms for element in innerList] # Convert it to a single list
            return flatList,synonym_dict 
        else:
            return r.json(),{}
    else:
        return [],{}
            

# May not use this function to avoid doing multiple requests for the same word
# Instead, use the aboive one and make one request per word, then search for the specific pos tag inside the returned dict
def getSynBigHugeSpec(complex_word, pos_tag):
    bighuge_synonyms = []
    r = requests.get(url='http://words.bighugelabs.com/api/2/'+BIGHUGE_KEY+'/'+complex_word+'/json') 
    if(r.status_code!=404):
        if(type(r.json()) is dict):
            synonym_dict = r.json()
            if(pos_tag in synonym_dict):
                bighuge_synonyms = synonym_dict[pos_tag].get("syn")
        else:
            bighuge_synonyms = r.json()
    return bighuge_synonyms

In [17]:
# OpenOffice thesaurus

# This generates a dictionary "thesaurus" with (word,pos) as the key and a set of synonyms as the value
# For example:
# thesaurus[("happy","adj")] = {'glad', 'pleased', 'prosperous', 'cheerful', ... }
# possible pos: noun,verb,adj,adv
def generateTheSaurusDict():
    thesaurus = {}
    with open("th_en_US_new.dat") as f:
        code = f.readline()    # Skip the file encoding
        while(True):
            word_count_line = f.readline()
            if(word_count_line == ""):
                break
            (word,count) = word_count_line.split('|')
            for i in range(0,int(count)):
                pos_synonyms = f.readline().split("|")
                synonyms_list = pos_synonyms[1:]
                pos = re.sub(r"[\([{})\]]", "",pos_synonyms[0]) # Remove te brackest surronding the pos (noun - verb - adv - adj)
                synonyms_list = [synonym.strip() for synonym in synonyms_list] # Remove unnecessary spaces
                if((word,pos) not in thesaurus):
                    thesaurus[(word,pos)]=set()
                for synonym in synonyms_list:
                    if(synonym not in thesaurus[(word,pos)] and synonym!=word):
                        thesaurus[(word,pos)].add(synonym)
    return thesaurus

# thesaurus = generateTheSaurusDict()
# print(thesaurus["happy","adj"])

In [18]:
# PPDB dictionary

def genPPDBdict(file_path):
    ppdb_dict = {}
    with open(file_path) as f:
        reader = csv.reader(f, delimiter="\t")
        for row in reader:
            line = row[0]
            word_list = line.split("|||")
            source = word_list[1].strip()
            target = word_list[2].strip()
            if source in ppdb_dict:
                ppdb_dict[source].append(target)
            else:
                ppdb_dict[source] = [target]
        return ppdb_dict
# with open("ppdb-2.0-tldr") as f:
#         dic = f.read()

In [19]:
# LS step2: Substitution Generation

thesaurus = generateTheSaurusDict()
ppdb = genPPDBdict(ppdb_filepath)

In [20]:
def genSubstitution(complex_word):
    thesaurus_candidates = []
    wordnet_candidates = []
    wordnet_candidates = getSynWordNet(complex_word)
    for key in thesaurus:
        if(key[0] == complex_word):
            thesaurus_candidates.append(thesaurus.get((key[0], key[1])))
    return wordnet_candidates, list(thesaurus_candidates)
    
def genSubstitutionSpec(complex_word, sentence):
    thesaurus_candidates = []
    bighuge_candidates = []
    wordnet_candidates = []
    ppdb_candidates = []
    pos_tag = getPosTagFromSentence(sentence, complex_word)
    word_type = getTypeFromTag(pos_tag)
    wordnet_candidates = getSynWordNetSpec(complex_word, word_type)
    flat_syn_list, bighuge_dict = [],{}
#     flat_syn_list, bighuge_dict = getSynBigHuge(complex_word)
    if(complex_word in ppdb):
        ppdb_candidates = ppdb[complex_word]
    if(flat_syn_list and not bighuge_dict):
        bighuge_candidates = flat_syn_list
    if(word_type == 'n'):
        thesaurus_candidates = thesaurus.get((complex_word,"noun"))
        if("noun" in bighuge_candidates):
            bighuge_candidates = bighuge_dict["noun"].get("syn")
    elif(word_type == 'r'):
        thesaurus_candidates = thesaurus.get((complex_word,"adv"))
        if("adverb" in bighuge_candidates):
            bighuge_candidates = bighuge_dict["adverb"].get("syn")
    elif(word_type == 'v'):
        thesaurus_candidates = thesaurus.get((complex_word,"verb"))
        if("verb" in bighuge_candidates):
            bighuge_candidates = bighuge_dict["verb"].get("syn") 
    elif(word_type == 'a'):
        thesaurus_candidates = thesaurus.get((complex_word,"adj"))
        if("adjective" in bighuge_candidates):
            bighuge_candidates = bighuge_dict["adjective"].get("syn")

    return wordnet_candidates, thesaurus_candidates, bighuge_candidates, ppdb_candidates

In [21]:
def filterSubstitutions(word_subs_dict):
    word_subs_dict_filtered = {}
    wnl = WordNetLemmatizer()
    ps = PorterStemmer()
    for word in word_subs_dict:
        word_lemm = wnl.lemmatize(word)
        filtered_subs_list = []
        subs_set = word_subs_dict[word]
        if(len(subs_set)>0):
            for subs in subs_set:
                word_list = nltk.word_tokenize(subs)
                word_split_hyphen = subs.split("-")
                word_split_underscore = subs.split("_")
                if(len(word_list) > 1):
                    add_word = True
                    for word_ in word_list:
                        word_lemm = wnl.lemmatize(word)
                        subs_lemm = wnl.lemmatize(word_)
                        if(ps.stem(word_)==ps.stem(word) or wnl.lemmatize(word_)==wnl.lemmatize(word) or is_similar(word_, word) or is_similar(word_, word_lemm) 
                          or is_similar(subs_lemm, word_lemm) or is_similar(subs_lemm, word)):
                            add_word = False
                            break
                    if(add_word):
                        filtered_subs_list.append(subs)
                elif(len(word_split_hyphen)>1):
                    add_word = True
                    for word_ in word_split_hyphen:
                        word_lemm = wnl.lemmatize(word)
                        subs_lemm = wnl.lemmatize(word_)
                        if(ps.stem(word_)==ps.stem(word) or wnl.lemmatize(word_)==wnl.lemmatize(word) or is_similar(word_, word) or is_similar(word_, word_lemm) 
                          or is_similar(subs_lemm, word_lemm) or is_similar(subs_lemm, word)):
                            add_word = False
                            break
                    if(add_word):
                        filtered_subs_list.append(subs)
                elif(len(word_split_underscore)>1):
                    add_word = True
                    for word_ in word_split_underscore:
                        word_lemm = wnl.lemmatize(word)
                        subs_lemm = wnl.lemmatize(word_)
                        if(ps.stem(word_)==ps.stem(word) or wnl.lemmatize(word_)==wnl.lemmatize(word) or is_similar(word_, word) or is_similar(word_, word_lemm)
                          or is_similar(subs_lemm, word_lemm) or is_similar(subs_lemm, word)):
                            add_word = False
                            break
                    if(add_word):
                        filtered_subs_list.append(subs)
                else: 
                    subs_lemm = wnl.lemmatize(subs)
                    if(ps.stem(subs)!=ps.stem(word) and wnl.lemmatize(subs)!=wnl.lemmatize(word) and not is_similar(subs, word) and not is_similar(subs, word_lemm)
                      and not is_similar(subs_lemm, word_lemm) and not is_similar(subs_lemm, word)):
                        filtered_subs_list.append(subs)
        word_subs_dict_filtered[word] = filtered_subs_list
    return word_subs_dict_filtered

In [70]:
def convertGrammStructure(word_subs_dict, word_sentence_dict):
    modified_word_subs_dict = word_subs_dict.copy()
    for word in word_subs_dict:
        subs_list = word_subs_dict[word]
        word_tag = getPosTagFromSentence(word_sentence_dict[word],word)
        word_type = getTypeFromTag(word_tag)
        if(word_type == 'n'): #NOUNS
            modified_subs = []
            for subs in subs_list:
                subs_tag = getPosTag(subs)
                subs_type = getTypeFromTag(subs_tag)
                if(word_tag=="NN" and subs_tag=="NNS"):
                    new_subs = singularize(subs)
                    modified_subs.append(new_subs)
                elif(word_tag=="NNS" and subs_tag=="NN"):
                    new_subs = pluralize(subs)
                    modified_subs.append(new_subs)
                elif((word_tag=="NNS" and subs_tag=="NNS") or (word_tag=="NN" and subs_tag=="NN")):
                    modified_subs.append(subs)
            modified_word_subs_dict[word] = modified_subs
        elif(word_type == 'a'): #ADJECTIVES
            print("adjj", word)
            modified_subs = []
            for subs in subs_list:
                subs_tag = getPosTag(subs)
                subs_type = getTypeFromTag(subs_tag)
                print(subs, subs_tag)
                if(word_tag=="JJR" and (subs_tag=="JJS" or subs_tag=="JJ")):
                    new_subs = comparative(subs)
                    modified_subs.append(new_subs)
                elif(word_tag=="JJS" and (subs_tag=="JJR" or subs_tag=="JJ")):
                    new_subs = superlative(subs)
                    modified_subs.append(new_subs)
                elif((word_tag=="JJR" and subs_tag=="JJR") or (word_tag=="JJS" and subs_tag=="JJS") or (word_tag=="JJ" and subs_tag=="JJ")):
                    modified_subs.append(subs)
            modified_word_subs_dict[word] = modified_subs
        elif(word_type == 'v'): #VERBS
            modified_subs = []
            try:
                complex_tense = tenses(word)
                for subs in subs_list:
                    subs_tokens = nltk.word_tokenize(subs)
    #                 subs_split = subs.split(" ")
                    subs_tag = getPosTag(subs)
                    subs_type = getTypeFromTag(subs_tag)
                    if(word_tag=="VBD" or word_tag=="VBN"): #-->PAST
                        if(len(subs_tokens)>1):
                            first_half_tense = tenses(subs)
                            if(subs_type=='v'):
                                new_word = conjugate(subs_tokens[0], tense=PAST)
                                subs_tokens[0] = new_word
                                new_subs = ' '.join(subs_tokens)
                                modified_subs.append(new_subs)
                        else:
                            new_subs = conjugate(subs, tense=PAST)
                            modified_subs.append(new_subs)
                    elif(word_tag=="VBP" or word_tag=="VBZ"): #-->PRESENT
                        new_subs = conjugate(subs, tense=PRESENT)
                        modified_subs.append(new_subs)
                    elif((len(complex_tense)>0 and complex_tense[0][0]=="infinitive")): #-->INFINITIVE
                        new_subs = conjugate(subs, tense=INFINITIVE)
                        modified_subs.append(new_subs)
                    elif(len(complex_tense)>0 and word_tag=="VBG"): #-->GERUND
                        new_subs = lemmatizer.lemmatize(subs) + "ing"
                        modified_subs.append(new_subs)
                    elif(len(complex_tense)>0 and complex_tense[0][0]=="future"): #-->FUTURE
    #                     print(word_tag, complex_tense, complex_tense[0][0], word, "\n")
                        new_subs = conjugate(subs, tense=FUTURE)
                        modified_subs.append(new_subs)   
                    else:
                        modified_subs.append(subs)  
                modified_word_subs_dict[word] = modified_subs
            except StopIteration as e:
                print("exceptionn") 
    return modified_word_subs_dict



In [63]:
# LS Step3: Substitution Ranking

def getNgramScore(phrase, start_year=2000, end_year=2019, corpus=26, smoothing=0):
    avg_score = 0
    google_ngram_url = "https://books.google.com/ngrams/json?content="+phrase+'&year_start=' + str(start_year) + '&year_end=' + str(end_year) + '&corpus=' + str(corpus) + '&smoothing=' + str(smoothing)
    response = requests.get(google_ngram_url)
    if(response):
        output = response.json()
#         print(output)
        return_data = []
        scores_list = []
        total_score = 0
        if(len(output) > 0):
            for num in range(len(output)):
                scores_list.append(output[num]['timeseries'])
            list_flatten = flatten(scores_list)
            for score in list_flatten:
                total_score+=(score)
            if(len(list_flatten) > 0):
                avg_score = total_score/len(list_flatten)
    return avg_score
            
def extractFeaturesFromWord(target_word, three_gram_dict):
    
    # Features we have are:
    # lex_exist_flag, complexity_score, word_length, syllable_count, wiki_freq, ngram_score
    lex_exist_flag = -1
    complexity_score = -1
    word_length = -1
    syllable_count = -1
    wiki_freq = -1
    ngram_score = -1

    
    # Before extracting features, check if it's a multi-word phrase, if so, we work on the longest word
    longest_word = ''
    word_list = nltk.word_tokenize(target_word)
    word_split_hyphen = target_word.split("-")
    word_split_underscore = target_word.split("_")
    if(len(word_list) > 1):
        longest_word = ''
        max_length = 0
        for word in word_list:
            if(len(word)>max_length):
                max_length=len(word)
                longest_word = word
    elif(len(word_split_hyphen) > 1):
        longest_word = ''
        max_length = 0
        for word in word_split_hyphen:
            if(len(word)>max_length):
                max_length=len(word)
                longest_word = word
    elif(len(word_split_underscore) > 1):
        longest_word = ''
        max_length = 0
        for word in word_split_underscore:
            if(len(word)>max_length):
                max_length=len(word)
                longest_word = word
        
    # EXTRACTING FEATURES
    
    # Feature 1: Binary number representing word's presence in lexicon (1:existent 0:non existent)
#     if(target_word in lexicon_dict):
#         lex_exist_flag = 1
#     else:
#         lex_exist_flag = 0
            
    # Feature 2: Complexity score of the word in the lexicon
    if(target_word in lexicon_dict):
        complexity_score = float(lexicon_dict[target_word])
    else:
        complexity_score = 0 # If word is not found in lexicon, set its complexity score with 0 also
      
    # Feature 3: word length (character count)
    word_length = len(target_word)
    
    # Feature 4: Syllable count
    syllable_count = syllables.estimate(target_word)     
    
    # Feature 5: Frequency with respect to Wiki-Frequency
    if(target_word in wiki_freq_dict):
        wiki_freq = int(wiki_freq_dict[target_word])
    else:
        wiki_freq = 0
        
    # Feature 6: Google Ngram average score
    if(target_word in three_gram_dict):
        three_words = three_gram_dict[target_word]
#         for word in three_words:
#             phrase+=word
        phrase = three_words[0] + " " + three_words[1] + " " + three_words[2]
#         print("p ",phrase)
        ngram_score = getNgramScore(phrase)
    else:
        ngram_score = 0
    
    return [complexity_score, word_length, syllable_count, wiki_freq, ngram_score] 

In [29]:
# Pair wise features

def getCosSim(vec_1, vec_2):
    cos_sim = cosine_similarity([vec_1], [vec_2])
    return cos_sim[0][0]

def similarityRatio(word1, word2):
    similarity_ratio = SequenceMatcher(None, word1, word2).ratio()
    return similarity_ratio 

In [30]:
def sigmoid(x):
    # TODO 1: Compute the sigmoid function at the given x (~1 line)
    # For example: sigmoid(2) should compute the value of sigmoid function at x = 2.
    # Hint: Use np.exp instead of math.exp to allow for vectorization.
    #----------------------------------------------------------------------------------------------
    sig = (1/(1+np.exp(-x)))
    #----------------------------------------------------------------------------------------------
    
    return sig

def batch_normalize(X, eps=1e-5):
    mean = np.mean(X, axis=0)
    var = np.var(X, axis=0)
    X_norm = (X - mean) / np.sqrt(var + eps)
    return X_norm

In [31]:
data = []
feature_matrix_ = []
cosine_similarities = []
similarity_ratios = []
candidates_matrix = []
sentence_list = []
word_candCount_dict = {}
three_gram_dict = {}
scores_matrix = []
complex_word_list = []
target_word = ''
X = []
num_features = 7
with open('BenchLS.txt') as f:
        data_reader = csv.reader(f, delimiter='\t')
        data = list(data_reader)
data = list(data)
# print(data)
train_count = math.ceil(0.8*len(data))
test_count = len(data) - train_count
train_data = data[:train_count]

num_candidates = 0
for line in train_data:
    candidates = []
    scores = []
    scores_candidates = line[3:]
    sentence = line[0]
    sentence_list.append(sentence)
    target_word = line[1]
    complex_word_list.append(target_word)
    num_candidates += len(scores_candidates)
    word_candCount_dict[target_word] = len(scores_candidates)
    for score_candidate in scores_candidates:
        candidates.append(score_candidate[2:])
        scores.append(int(score_candidate[0]))
    candidates_matrix.append(candidates)
    scores_matrix.append(scores)

indx = 0
for line in candidates_matrix:
    sentence = sentence_list[indx]
    complex_word = complex_word_list[indx]
    current_candidates_features = []
    for candidate in line:
        prev_word = word_preceding(sentence.lower(), complex_word)
        next_word = word_following(sentence.lower(), complex_word)
        if(prev_word and next_word):
            three_gram_dict[candidate] = [prev_word, candidate, next_word]
        feature_list_ = extractFeaturesFromWord(candidate,three_gram_dict)
        current_candidates_features.append(feature_list_)
        if(target_word in word_vectors and candidate in word_vectors):     
            target_word_vector = word_vectors[target_word]
            substitution_vector = word_vectors[candidate]
            cos_similarity = getCosSim(target_word_vector, substitution_vector)
        else:
            cos_similarity = 0
        cosine_similarities.append(cos_similarity)
        similarity_ratios.append(similarityRatio(target_word, candidate))
    current_candidates_features = np.array(current_candidates_features).reshape(len(line),num_features-2)
    max_in_column = np.max(current_candidates_features,axis=0)
    for i in range(num_features-2):
        if(max_in_column[i] != 0):
            current_candidates_features[:, i] = current_candidates_features[:, i]/max_in_column[i]
    feature_matrix_.append(current_candidates_features)
#     print(feature_matrix_)
    indx+=1

feature_matrix_ = np.concatenate(feature_matrix_, axis=0)
cosine_similarities = np.array(cosine_similarities).reshape(num_candidates,1)
similarity_ratios = np.array(similarity_ratios).reshape(num_candidates,1)

X = np.hstack((feature_matrix_,cosine_similarities, similarity_ratios))

Y = np.ones((len(feature_matrix_),1))
y_indx = 0
max_score = max(flatten(scores_matrix))
for line in scores_matrix: 
    for score in line:
        Y[y_indx]=score/max_score
        y_indx+=1

m = len(X)  # training set size
nn_input_dim = 7  # input layer dimensionality (we have seven input features)
nn_output_dim = 1  # output layer dimensionality (we have one output :: score)

# Gradient descent parameters
alpha = 0.1 # learning rate for gradient descent

def buildModel(nn_hdim, num_passes=20000, print_loss=False):

    np.random.seed(0)
    W1 = np.random.randn(nn_hdim, nn_input_dim) / np.sqrt(nn_input_dim)
    b1 = np.zeros((nn_hdim, 1))
    W2 = np.random.randn(nn_output_dim, nn_hdim) / np.sqrt(nn_hdim)
    b2 = np.zeros((nn_output_dim, 1))
    
    model = {}

    for i in range(0, num_passes):
        DW1 = 0
        DW2 = 0
        Db1 = 0
        Db2 = 0
        cost = 0

        for j in range(0, m):
            a0 = X[j, :].reshape(-1, 1) 
            y = Y[j]
            
            # Forward propagation
            z1 = np.dot(W1 , a0 )+ b1
            a1 = np.tanh(z1)
            z2 = np.dot(W2 , a1) + b2
            a2 = z2
#             print("z11", z1, "\n")
#             print("a11", a1, "\n")
#             print("W2", W2, "\n")
#             print("b2", b2, "\n")
#             print("z2 = W2.a1 + b2", z2, "\n")

#             print("W4", W4, "\n")
#             print("a3", a3, "\n")
#             print("b4", b4, "\n")
#             print("z4 = W4.a3 + b4", z4, "\n")


            cost_j = abs(y-a2)
       
            da2 =  ( -y/a2  + (1-y)/(1-a2) )
            dz2 =  da2 * a2 * ( 1 - a2)
            dW2 = np.dot(dz2 , a1.T)
            db2 = dz2

            da1 =  np.dot(dz2,W2).T
            dz1 = np.multiply(da1 , 1 - np.square(a1) )
            dW1 = np.dot(dz1 , a0.T )
            db1 = dz1

            DW1 += dW1
            DW2 += dW2
            Db2 += db2
            Db1 += db1
            cost += cost_j
            
        # Averaging DW1, DW2, Db1, Db2 and cost over the m training examples. 
        DW1 /= m
        DW2 /= m
        Db1 /= m
        Db2 /= m
        cost /= m

        # Gradient descent parameter update
        W1 -= alpha * DW1
        b1 -= alpha * Db1
        W2 -= alpha * DW2
        b2 -= alpha * Db2

        # Assign new parameters to the model
        model = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}

        if print_loss and i % 100 == 0:
            print("Loss after iteration %i: %f" % (i, cost))

    return model


p  missing parts was
p  missing bits was
p  missing components was
p  missing information was
p  missing elements was
p  missing items was
p  missing component was
p  missing part was
p  missing sections was
p  is redirected .
p  is rerouted .
p  is changed .
p  is moved .
p  is separated .
p  is turned .
p  is altered .
p  is split .
p  is switched .
p  is veered .
p  is channeled .
p  is deflected .
p  worthy owner of
p  worthy holder of
p  worthy keeper of
p  worthy buyer of
p  worthy master of
p  worthy teacher of
p  other dangers ,
p  other hazards ,
p  other situations ,
p  other danger ,
p  other problems ,
p  other fates ,
p  other difficulties ,
p  other risks ,
p  other restraints ,
p  other troubles ,
p  and kept its
p  and held its
p  and maintained its
p  and regained its
p  official home of
p  official house of
p  official dwelling of
p  official abode of
p  official owner of
p  official people of
p  is obtained with
p  is gathered with
p  is gotten with
p  is grabbed wit

p  who let the
p  who gave the
p  a thin canapy
p  a small canapy
p  a limited canapy
p  a few canapy
p  a scattered canapy
p  a skimpy canapy
p  a light canapy
p  a bare canapy
p  a scant canapy
p  a little canapy
p  a slim canapy
p  how rewards for
p  how motivations for
p  how bonuses for
p  how awards for
p  how benefits for
p  how motives for
p  how bribes for
p  how prizes for
p  how motivation for
p  how bait for
p  how gift for
p  how stimulants for
p  how encouragements for
p  how goals for
p  how baits for
p  how temptations for
p  how boost for
p  how payments for
p  an tool that
p  an instrument that
p  an device that
p  an item that
p  not support coercion
p  not promote coercion
p  not suggest coercion
p  not endorse coercion
p  not allow coercion
p  not like coercion
p  not for coercion
p  not state coercion
p  not favor coercion
p  not encourage coercion
p  not want coercion
p  not recommend coercion
p  not apply coercion
p  not cause coercion
p  not speak coercion
p  i

p  to avoid capture
p  to escape capture
p  to dodge capture
p  to elude capture
p  to not capture
p  to be capture
p  a village of
p  a town of
p  a community of
p  a suburb of
p  a division of
p  a neighborhood of
p  a district of
p  a house of
p  a place of
p  a small of
p  an different universe
p  an other universe
p  an another universe
p  an opposite universe
p  an parallel universe
p  an separate universe
p  an imaginary universe
p  an recurrent universe
p  an variant universe
p  an varied universe
p  an mirror universe
p  them appropriate for
p  them acceptable for
p  them good for
p  them useful for
p  them okay for
p  them fitting for
p  them usable for
p  them apt for
p  them fit for
p  them ideal for
p  them perfect for
p  them convenient for
p  them applicable for
p  them excellent for
p  her sick mother
p  her ill mother
p  her sickly mother
p  her dying mother
p  her suffering mother
p  her distressed mother
p  her sickening mother
p  economic problems throughout
p  econ

p  rich colored hue
p  the rebirth of
p  the return of
p  the comeback of
p  the renewal of
p  the awakening of
p  the recovery of
p  the restart of
p  the reuse of
p  the comback of
p  the reemergence of
p  the reintroduction of
p  the reformation of
p  the remake of
p  the restoration of
p  in southern france
p  in lower france
p  in southwest france
p  in south france
p  in south-west france
p  in lower-left france
p  in part france
p  in sw france
p  in western france
p  in bottom france
p  a hitter .
p  a forward .
p  a player .
p  a attacker .
p  a protestor .
p  a protester .
p  a scorer .
p  a looker .
p  a trouble-maker .
p  a worker .
p  a sportsman .
p  a batter .
p  a kicker .
p  a demonstrator .
p  a foreward .
p  a footballer .
p  a picketer .
p  also said that
p  also stated that
p  also wrote that
p  also noted that
p  also told that
p  also mentioned that
p  also claimed that
p  also declared that
p  also announced that
p  also indicated that
p  benson worked with
p  b

p  the center of
p  the office of
p  the capital of
p  the hometown of
p  the seat of
p  is thought to
p  is known to
p  is accepted to
p  is said to
p  is understood to
p  is trusted to
p  italian former footballer
p  italian resigned footballer
p  italian old footballer
p  italian ex footballer
p  italian past footballer
p  italian inactive footballer
p  italian elderly footballer
p  italian finished footballer
p  italian withdrawn footballer
p  italian one-time footballer
p  italian previous footballer
p  in tree groves
p  in pine groves
p  in evergreen groves
p  in fir groves
p  in wood groves
p  in cedar groves
p  in grassy groves
p  in nice groves
p  being removed .
p  being pushed .
p  being expelled .
p  being ejected .
p  being squeezed .
p  being shaped .
p  being forced .
p  being made .
p  being extracted .
p  being thrusted .
p  being formed .
p  being dismissed .
p  being erected .
p  being held .
p  being examined .
p  being pressed .
p  being fixed .
p  being taken .
p 

...

p  they moved them
p  they took them
p  they declined them
p  they changed them
p  are credited to
p  are attributed to
p  are referred to
p  are assigned to
p  are given to
p  are tied to
p  are acredited to
p  are applied to
p  are refered to
p  are associated to
p  are explained to
p  are due to
p  are recognized to
p  are specific to
p  are asigned to
p  are connected to
p  are accredited to
p  are jealous to
p  unique color .
p  unique shade .
p  unique tone .
p  particularly meaningful .
p  particularly important .
p  particularly memorable .
p  particularly powerful .
p  particularly fitting .
p  particularly thrilling .
p  particularly appealing .
p  particularly deep .
p  particularly profound .
p  particularly resounding .
p  particularly satisfying .
p  particularly impressive .
p  particularly interesting .
p  particularly heard .
p  particularly relatable .
p  particularly moving .
p  particularly rich .
p  particularly vibrant .
p  particularly known .
p  particularly mea

p  she observed the
p  she seen the
p  she noticed the
p  which has an
p  which shows an
p  which includes an
p  which displays an
p  which boasts an
p  which offers an
p  which hallmarks an
p  which spotlights an
p  observed mostly by
p  observed mainly by
p  observed normally by
p  observed exclusively by
p  observed predominantly by
p  it is at
p  it sits at
p  it exists at
p  it rests at
p  it locates at
p  it sets at
p  it stands at
p  it resides at
p  it stations at
p  catastrophic explosion of
p  catastrophic blast of
p  catastrophic burst of
p  catastrophic blasting of
p  catastrophic ejection of
p  catastrophic spuing of
p  catastrophic blow-up of
p  catastrophic explosion' of
p  catastrophic outburst of
p  are seldom found
p  are hardly found
p  are uncommonly found
p  are barely found
p  are sometimes found
p  was told to
p  was taught to
p  was ordered to
p  to add to
p  to give to
p  to cause to
p  to lend to
p  to provide to
p  to assist to
p  to share to
p  to created to

p  widely believed the
p  widely admired the
p  widely deemed the
p  widely called the
p  widely chosen the
p  widely regarded the
p  widely reviewed the
p  widely known the
p  widely crowned the
p  widely agreed the
p  widely viewed the
p  and lasted another
p  and lived another
p  and remained another
p  and raged another
p  and existed another
p  and continued another
p  in reply to
p  in answer to
p  in reaction to
p  in regards to
p  in looking to
p  in answering to
p  in return to
p  in expectation to
p  in acknowledgement to
p  it hit a
p  it gained a
p  it topped a
p  it made a
p  it received a
p  it extended a
p  it held a
p  it achieved a
p  it acheived a
p  it met a
p  it touched a
p  it attained a
p  it got a
p  it entered a
p  it was a
p  city found in
p  city situated in
p  city placed in
p  city based in
p  city set in
p  city laying in
p  city is in
p  city positioned in
p  city that has it's place in
p  city lying in
p  city nearby in
p  in specific the
p  in especiall

p  and consequently have
p  season so far
p  season this far
p  can therefore be
p  can accordingly be
p  can consequently be
p  imperative therefore confronts
p  imperative consequently confronts
p  are therefore thought
p  are hence thought
p  are consequently thought
p  is therefore likely
p  is accordingly likely
p  is consequently likely
p  season so far
p  is therefore likely
p  is accordingly likely
p  is consequently likely
p  and there begins
p  and so begins
p  and consequently begins
p  and therefore begins
p  : private trial
p  : citizen trial
p  : noncriminal trial
p  : civic trial
p  our elected authorities
p  our public authorities
p  our state authorities
p  our civic authorities
p  of state law
p  of noncriminal law
p  of civic law
p  relatively polite and
p  relatively cordial and
p  relatively courteous and
p  relatively inoffensive and
p  from internal wars
p  from people\'s wars
p  from domestic wars
p  , polite society
p  , public society
p  , general society
p  ,

p  very quickly without
p  very briefly without
p  very sharply without
p  very abruptly without
p  very curtly without
p  very brusquely without
p  you soon come
p  you quickly come
p  you presently come
p  becca sharply ,
p  becca abruptly ,
p  becca curtly ,
p  becca presently ,
p  fraser soon before
p  fraser just before
p  can thus be
p  can accordingly be
p  can hence be
p  can consequently be
p  and so produces
p  and thus produces
p  and consequently produces
p  terms thus seek
p  terms accordingly seek
p  terms consequently seek
p  and so they
p  and thus they
p  and consequently they
p  is thus instrumental
p  is accordingly instrumental
p  is consequently instrumental
p  should thus report
p  should accordingly report
p  should consequently report
p  we thus oppose
p  we accordingly oppose
p  we consequently oppose
p  and hence ,
p  and accordingly ,
p  and thus ,
p  and consequently ,
p  will hence permit
p  will accordingly permit
p  will consequently permit
p  and so part

In [41]:
model = buildModel(8,10001,True)
print(model)

Loss after iteration 0: 0.503491
Loss after iteration 100: 0.182537
Loss after iteration 200: 0.170546
Loss after iteration 300: 0.167049
Loss after iteration 400: 0.165432
Loss after iteration 500: 0.164535
Loss after iteration 600: 0.163987
Loss after iteration 700: 0.163627
Loss after iteration 800: 0.163378
Loss after iteration 900: 0.163196
Loss after iteration 1000: 0.163061
Loss after iteration 1100: 0.162956
Loss after iteration 1200: 0.162871
Loss after iteration 1300: 0.162801
Loss after iteration 1400: 0.162743
Loss after iteration 1500: 0.162694
Loss after iteration 1600: 0.162653
Loss after iteration 1700: 0.162616
Loss after iteration 1800: 0.162584
Loss after iteration 1900: 0.162554
Loss after iteration 2000: 0.162528
Loss after iteration 2100: 0.162504
Loss after iteration 2200: 0.162481
Loss after iteration 2300: 0.162461
Loss after iteration 2400: 0.162441
Loss after iteration 2500: 0.162424
Loss after iteration 2600: 0.162407
Loss after iteration 2700: 0.162392
Loss

In [42]:
def predict(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    a0 = x.T

    # Forward propagation
    z1 = np.dot(W1 , a0) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(W2 , a1) + b2
    a2 = z2

    prediction = a2
    
#     print("X ", x, "\n")
#     print("z11", z1, "\n")
#     print("a11", a1, "\n")
#     print("W2", W2, "\n")
#     print("b2", b2, "\n")
#     print("z2 = W2.a1 + b2", z2, "\n")
    
    return prediction[0]

In [71]:
def simplify(text):
    num_features = 7
    vowels = "aeiouAEIOU"
    prediction = []
    word_replace_dict = {}
    thresh_scores = {} 
    thresh_scores, word_sentence_dict = complexWordIdentif(text)
    word_subst_dict = {}
    new_text = text # initialize the new string with the original one
    for word in word_sentence_dict:
        word_subst_dict[word] = set()
        sentence = word_sentence_dict[word]
        word_net_cand, thesaurus_cand, bighuge_cand, ppdb_candidates = genSubstitutionSpec(word, sentence)
        if(word_net_cand):
            word_subst_dict[word].update(word_net_cand)
        if(thesaurus_cand):
            word_subst_dict[word].update(thesaurus_cand)
        if(bighuge_cand):
            word_subst_dict[word].update(bighuge_cand)
        if(ppdb_candidates):
            word_subst_dict[word].update(ppdb_candidates)
    filtered_subs_dict = filterSubstitutions(word_subst_dict)
#     print(word_subst_dict)
#     print(filtered_subs_dict)
    modified_word_subs_dict = convertGrammStructure(filtered_subs_dict, word_sentence_dict)    
#     print(modified_word_subs_dict)
    
    # EXTRACT FEATURES
    for target_word in modified_word_subs_dict:
        three_gram_dict = {}
        feature_matrix = []
        cosine_similarities = []
        similarity_ratios = []
        candidates = []
        candidates = modified_word_subs_dict[target_word]
#         print(candidates)
#         candidates = [target_word] + modified_word_subs_dict[target_word]
        if(len(candidates)>0):
            for candidate in candidates:
                three_gram_phrase = ''
                prev_word = word_preceding(sentence, target_word)
                next_word = word_following(sentence, target_word)
                if(prev_word and next_word):
                    three_gram_phrase = prev_word + " " + candidate + " " + next_word
                    three_gram_dict[candidate] = [prev_word, candidate, next_word]
                features_list = extractFeaturesFromWord(candidate, three_gram_dict)
                if(target_word in word_vectors and candidate in word_vectors):     
                    target_word_vector = word_vectors[target_word]
                    substitution_vector = word_vectors[candidate]
                    cos_similarity = getCosSim(target_word_vector, substitution_vector)
                else:
                    cos_similarity = 0
                similarity_ratios.append(similarityRatio(target_word, candidate))
                cosine_similarities.append(cos_similarity)
                feature_matrix.append(features_list)
            cosine_similarities = np.array(cosine_similarities).reshape(len(feature_matrix),1)
            similarity_ratios = np.array(similarity_ratios).reshape(len(feature_matrix),1)
            X = np.hstack((feature_matrix,cosine_similarities, similarity_ratios))
            max_in_column = np.max(X,axis=0)
            for i in range(num_features):
                if(max_in_column[i] != 0):
                    X[:, i] /= max_in_column[i]
#             print(candidates)
#             print(X)
            prediction = predict(model, X)
#             print(candidates)
#             print(prediction)
            min_value = min(prediction)
            prediction_list = prediction.tolist()
            min_index=prediction_list.index(min_value)
#             print(prediction_list,"\n",candidates)
            chosen_candidate = candidates[min_index]
            if(prev_word == 'a' and chosen_candidate[0] in vowels):
                new_text = new_text.replace('a '+ target_word, 'an ' + chosen_candidate)
            elif(prev_word == 'an' and chosen_candidate[0] not in vowels):
                new_text = new_text.replace('an '+ target_word, 'a ' + chosen_candidate)
            else:
                new_text = new_text.replace(target_word, chosen_candidate)
            word_replace_dict[target_word] = chosen_candidate
#             print(candidates, "\n", prediction)
    return new_text

In [76]:
original = 'the emancipation of women can only be completed when a fundamental transformation of living is effected ; and life-styles will change only with the fundamental transformation of all production and the establishment of a communist economy ' 
simplified = simplify(original)
print(original)
print(simplified)
# print(getPosTagFromSentence('This place is astounding' , "astounding"))

adjj fundamental
foundations NNS
rudimentary JJ
vital NN
important JJ
cardinal NN
keys NNS
essentials NNS
profound NN
essential JJ
crucial JJ
key NN
of import NN
underlying VBG
basic JJ
central JJ
primal NN
der NN
significant JJ
adjj communist
political orientation NN
ideology NN
political theory NN
the emancipation of women can only be completed when a fundamental transformation of living is effected ; and life-styles will change only with the fundamental transformation of all production and the establishment of a communist economy 
the release of women can only be completed when a significant change of living is created ; and life-styles will change only with the significant change of all act and the construction of a communist economy 
